In [7]:
import numpy as np
import math
import heapq

def manhattanDistance(x,y):  
    """L1范式下欧几里得空间距离<Manhattan距离>计算
    INPUT: 点x与点y维数相同
    x: list or set or tuple. 点x
    y: list or set or tuple. 点y
    
    OUTPUT:点x与点y的Manhattan距离
    """
    return sum(map(lambda i,j:abs(i-j),x,y))

def kClosestSort(points,k,origin):
    """返回离点origin在L1范式下欧几里得空间距离最靠近的k个点（堆排序） 时间复杂度Θ(nlogn)
    INPUT:
    points: list. 点集
    k: int. 返回前k个元素
    origin: list or set or tuple.
    
    OUTPUT:最靠近origin的k个点
    """
    return heapq.nsmallest(k, points, lambda p:manhattanDistance(p,origin) )

def select_kth(A,low,high,k,q):
    """选择第k近元素 时间复杂度Θ(n)
    INPUT:
    A：list. 点集
    low: int. 起始位置
    high: int. 结束位置
    k: int. 第k个
    q: list or set or tuple. 参照点
    
    OUTPUT:A中里点q第k近元素
    """ 
    p=high-low+1
    if p<44:
        A=kClosestSort(A,p,q)
        return (A[low+k-1])
    t=int(math.floor(p/5))
    M=list()
    temp=list()
    for i in range(5*t): #5个一组取中间元素
        temp.append(A[i])
        if i%5==4:
            temp=kClosestSort(temp,5,q)
            M.append(temp[3])
            temp.clear()
    mm=select_kth(M,0,t-1,int(math.ceil(t/2)),q)
    A1=list()
    A2=list()
    A3=list()
    for a in A:
        if manhattanDistance(a,q)<manhattanDistance(mm,q):
            A1.append(a)
        elif manhattanDistance(a,q)==manhattanDistance(mm,q):
            A2.append(a)
        else:
            A3.append(a)
    lenA1=len(A1)
    lenA2=len(A2)
    lenA3=len(A3)
    if lenA1>=k:
        return select_kth(A1,0,lenA1-1,k,q)
    if lenA1+lenA2>=k:
        return mm
    if lenA1+lenA2<k:
        return select_kth(A3,0,lenA3-1,k-lenA1-lenA2,q)

    
def LSH_Init(matrix,hash_num,C):
    """LSH预处理
    INPUT:
    matrix: array. 点集P
    hash_num: int. 哈希表数量L
    C: int. 点集中最大坐标
    
    OUTPUT：哈希函数簇 hash_tables，哈希表1...L bucket_tables(论文中是写入硬盘)
    """
    d=len(matrix[0])  # 维度
    mlen=70  # hash长度
    n=len(matrix) # 点集大小
    #alpha=2 
    #B=10  # 每个桶的大小
    #M=alpha*(n//B) # 每个哈希表的大小
    hash_tables=np.random.randint(0,C*d,[hash_num,mlen])  ##调参
    bucket_tables=dict()   #对哈希表1...L的映射
    for i in range(len(hash_tables)): #哈希表1...L
        buckets=dict()     #该哈希表i
        for j in range(n): #遍历点集
            temp=list()
            #if len(buckets)==M:  哈希表大小超过M
                #break
            for m in hash_tables[i]: #每一位（下标从0开始）
                temp_1=m//C #对应原数据中第几位坐标
                temp_2=m%C  #对应上述坐标中在汉明空间下指定的位数
                if temp_2<matrix[j][temp_1]:
                    temp.append(1)
                else:
                    temp.append(0)
            label=tuple(temp)
            if label not in buckets.keys():
                buckets[label]=set()    #分配一个新桶
            #if len(buckets[label])<B:  #该桶的容量小于B
            buckets[label].add(tuple(matrix[j]))  #点入桶
        bucket_tables[i]=buckets        #第i个哈希表加入映射
    return hash_tables,bucket_tables

def LSH_Quire(q,k,hash_tables,bucket_tables,C):
    """近似临近查询
    INPUT:
    q：list or tuple. 查询点
    k：int. 查询最近邻数
    hash_tables： 哈希函数簇
    bucket_tables: 哈希表1...L
    
    OUTPUT: list. K个最近邻点
    """
    k_points=set()
    for i in range(len(hash_tables)):
        temp=list()
        for m in hash_tables[i]:
            temp_1=m//C #对应原数据中第几位坐标
            temp_2=m%C  #对应上述坐标中在汉明空间下指定的位数
            if temp_2<q[temp_1]:
                temp.append(1)
            else:
                temp.append(0)
        label=tuple(temp)
        if label in bucket_tables[i].keys():
            k_points=k_points|bucket_tables[i][label]   #并集S
    points=list(k_points)
    kth_point=select_kth(points,0,len(points)-1,k,q)  #求第k近元素 kth_point
    """扫描S返回最靠近点q的K个点"""
    result=list()
    temp=list()
    for point in points:
        if manhattanDistance(point,q)<manhattanDistance(kth_point,q): #到查询点q距离小于第k近元素
            result.append(point)
        elif manhattanDistance(point,q)==manhattanDistance(kth_point,q): #到查询点q距离等于第k近元素
            temp.append(point)
    if len(result)<k:  #缺k-len(result)元素满k个，用到第k近元素补满到k
        for i in range(k-len(result)):
            result.append(temp[i])
    return result

def Brute_ForceLinearSearch(q,matrix,k):
    """线性暴力搜索k近点
    INPUT：
    q：list or tuple or set. 查询点q
    matrix: array. 点集P
    k：int. 查询最近邻数
    
    OUTPUT: list. K个最近邻点
    """
    real_temp=matrix.tolist()
    real_kth_point=select_kth(real_temp,0,len(real_temp)-1,k,q)
    real=list()  #真实结果P'
    temp=list()  #存第k近的点
    for point in real_temp:
        if manhattanDistance(point,q)<manhattanDistance(real_kth_point,q): #到查询点q距离小于第k近元素
            real.append(tuple(point))
        elif manhattanDistance(point,q)==manhattanDistance(real_kth_point,q): #到查询点q距离等于第k近元素
            temp.append(tuple(point))
    if len(real)<k:  #缺k-len(result)元素满k个，用第k近元素补满到k
        for i in range(k-len(real)):
            real.append(tuple(temp[i]))
    return real

def LSH_Error(Pk,Qk,q):
    """论文中的error计算
    INPUT：
    Pk：list or tuple. 暴力线性搜索查询结果点集（真实结果）
    Qk：list or tuple. LSH查询结果点集
    q：list or tuple or set. 查询点q
    
    OUTPUT: 误差率
    """
    k=len(Qk)
    Pk=kClosestSort(Pk,k,q)
    Qk=kClosestSort(Qk,k,q)
    dsum=0
    for i in range(k):
        lshdist=manhattanDistance(Qk[i],q)
        mindist=manhattanDistance(Pk[i],q)
        dsum+=lshdist/mindist
    error=dsum/k
    return error

def Recall(real_k_points,lsh_k_points):
    """求LSH算法召回率
    INPUT:
    real_k_points: list. 真实k近点集P'
    lsh_k_points： list. 查询得到点集Q
    
    OUTPUT：召回率 |P'∩Q|/k
    """
    k=len(real_k_points)
    recall=len(list(set(real_k_points).intersection(set(lsh_k_points))))/k
    return recall
      
    
    
hash_num=10  #hash表数量
k=100 #最近邻点数
C=100 #点集中最大坐标
matrix=np.random.randint(1,C,[270000,5]) #生成点集
hash_tables,bucket_tables=LSH_Init(matrix,hash_num,C)
q=np.random.randint(1,100,[1,5]) #随机生成查询点
q=tuple(q[0]) #获取查询点
print('查询点: '+str(q))
Qk=LSH_Quire(q,k,hash_tables,bucket_tables,C)
Pk=Brute_ForceLinearSearch(q,matrix,k)
#error=LSH_Error(Pk,Qk,q)
#print('error: '+str(error))
recall=Recall(Pk,Qk)
print('recall: '+str(recall))

查询点: (99, 16, 62, 60, 24)


IndexError: list index out of range

In [14]:
import numpy as np
import math
import heapq

def manhattanDistance(x,y):  
    return sum(map(lambda i,j:abs(i-j),x,y))

def kClosestSort(points,k,origin):
    return heapq.nsmallest(k, points, lambda p:manhattanDistance(p,origin) )

def select_kth(A,low,high,k,q):
    p=high-low+1
    if p<44:
        A=kClosestSort(A,p,q)
        return (A[low+k-1])
    t=int(math.floor(p/5))
    M=list()
    temp=list()
    for i in range(5*t): #5个一组取中间元素
        temp.append(A[i])
        if i%5==4:
            temp=kClosestSort(temp,5,q)
            M.append(temp[3])
            temp.clear()
    mm=select_kth(M,0,t-1,int(math.ceil(t/2)),q)
    A1=list()
    A2=list()
    A3=list()
    for a in A:
        if manhattanDistance(a,q)<manhattanDistance(mm,q):
            A1.append(a)
        elif manhattanDistance(a,q)==manhattanDistance(mm,q):
            A2.append(a)
        else:
            A3.append(a)
    lenA1=len(A1)
    lenA2=len(A2)
    lenA3=len(A3)
    if lenA1>=k:
        return select_kth(A1,0,lenA1-1,k,q)
    if lenA1+lenA2>=k:
        return mm
    if lenA1+lenA2<k:
        return select_kth(A3,0,lenA3-1,k-lenA1-lenA2,q)

    
def LSH_Init(matrix,hash_num,C):
    d=len(matrix[0])  # 维度
    mlen=5  # hash长度
    n=len(matrix) # 点集大小
    #alpha=2 
    #B=10  # 每个桶的大小
    #M=alpha*(n//B) # 每个哈希表的大小
    hash_tables=np.random.randint(0,C*d,[hash_num,mlen])  ##调参
    bucket_tables=dict()   #对哈希表1...L的映射
    for i in range(len(hash_tables)): #哈希表1...L
        buckets=dict()     #该哈希表i
        for j in range(n): #遍历点集
            temp=list()
            #if len(buckets)==M:  哈希表大小超过M
                #break
            for m in hash_tables[i]: #每一位（下标从0开始）
                temp_1=m//C #对应原数据中第几位坐标
                temp_2=m%C  #对应上述坐标中在汉明空间下指定的位数
                if temp_2<matrix[j][temp_1]:
                    temp.append(1)
                else:
                    temp.append(0)
            label=tuple(temp)
            if label not in buckets.keys():
                buckets[label]=set()    #分配一个新桶
            #if len(buckets[label])<B:  #该桶的容量小于B
            buckets[label].add(tuple(matrix[j]))  #点入桶
        bucket_tables[i]=buckets        #第i个哈希表加入映射
    return hash_tables,bucket_tables


def LSH_Quire(quires,K,hash_tables,bucket_tables,C):
    qnum=len(quires)
    result=dict()
    for q in range(qnum):
        S=set()
        for i in range(len(hash_tables)):
            temp=list()
            for m in hash_tables[i]:
                temp_1=m//C #对应原数据中第几位坐标
                temp_2=m%C  #对应上述坐标中在汉明空间下指定的位数
                if temp_2<quires[q][temp_1]:
                    temp.append(1)
                else:
                    temp.append(0)
            label=tuple(temp)
            if label in bucket_tables[i].keys():
                S=S|bucket_tables[i][label]   #并集S
        S=list(S)
        SK=list()
        if len(S)>K:
            kth_point=select_kth(S,0,len(S)-1,K,quires[q])  #求第k近元素 kth_point
            """扫描S返回最靠近点q的K个点"""
            temp=list()
            for point in S:
                if manhattanDistance(point,quires[q])<manhattanDistance(kth_point,quires[q]): #到查询点q距离小于第k近元素
                    SK.append(point)
                elif manhattanDistance(point,quires[q])==manhattanDistance(kth_point,quires[q]): #到查询点q距离等于第k近元素
                    temp.append(point)
            if len(SK)<K:  #缺k-len(SK)元素满K个，用到第k近元素补满到K
                for i in range(K-len(SK)):
                    SK.append(temp[i])
        else:
            SK=S
        result[q]=SK
    return result


def Brute_ForceLinearSearch(quires,matrix,K):
    """线性暴力搜索k近点"""
    data=matrix.tolist()
    qnum=len(quires)
    real=dict()
    for q in range(qnum):
        real_kth_point=select_kth(data,0,len(data)-1,K,quires[q])
        q_real=list()  #真实结果P'
        temp=list()  #存第k近的点
        for point in data:
            if manhattanDistance(point,quires[q])<manhattanDistance(real_kth_point,quires[q]): #到查询点q距离小于第k近元素
                q_real.append(tuple(point))
            elif manhattanDistance(point,quires[q])==manhattanDistance(real_kth_point,quires[q]): #到查询点q距离等于第k近元素
                temp.append(tuple(point))
        if len(q_real)<K:  #缺k-len(result)元素满k个，用第k近元素补满到k
            for i in range(K-len(q_real)):
                q_real.append(tuple(temp[i]))
        real[q]=q_real
    return real

def LSH_Error(Pk,Qk,q):
    k=len(Qk)
    Pk=kClosestSort(Pk,k,q)
    Qk=kClosestSort(Qk,k,q)
    dsum=0
    for i in range(k):
        lshdist=manhattanDistance(Qk[i],q)
        mindist=manhattanDistance(Pk[i],q)
        dsum+=lshdist/mindist
    error=dsum/k
    return error


def Recall(real_k_points,lsh_k_points):
    qnum=len(real_k_points)
    temp1=0
    temp2=0
    for q in range(qnum):
        K=len(real_k_points[q])
        temp1+=len(list(set(lsh_k_points[q]).intersection(set(real_k_points[q]))))
        temp2+=K
    recall=temp1/temp2
    return recall      
    
    
hash_num=20  #hash表数量
K=1 #最近邻点数
C=20 #点集中最大坐标
#matrix=np.random.randint(1,C,[19000,20]) #生成点集
#quires=np.random.randint(1,100,[100,20]) #随机生成查询点
#np.save('LSH_Quires',quires)
#np.save('LSH_DataSet',matrix)
matrix=np.load('LSH_DataSet.npy')
quires=np.load('LSH_Quires.npy')
hash_tables,bucket_tables=LSH_Init(matrix,hash_num,C)

#q=tuple(quires[0]) #获取查询点
#print('查询点: '+str(q))
Qk=LSH_Quire(quires,K,hash_tables,bucket_tables,C)
Pk=Brute_ForceLinearSearch(quires,matrix,K)
#error=LSH_Error(Pk,Qk,q)
#print('error: '+str(error))
recall=Recall(Pk,Qk)
print('recall: '+str(recall))

recall: 0.93
